In [0]:
# Databricks Notebook: Transform Bronze (FNG) -> Silver (fixed w/ ts_unix)
from pyspark.sql import functions as F
from delta.tables import DeltaTable

spark.sql("SET spark.sql.session.timeZone=UTC")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
BRONZE  = f"{CATALOG}.{SCHEMA}.bronze_fear_greed"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_fear_greed"

# Silver 스키마(호환용으로 ts_unix 포함)
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {SILVER} (
  ts_unix              BIGINT,      -- event_time(초) 파생
  event_time           TIMESTAMP,   -- 원본 UTC 시각
  dt                   DATE,        -- 파티션
  index_value          INT,         -- 0~100
  value_classification STRING,      -- "Extreme Fear" ~ "Extreme Greed"
  time_until_update    STRING,      -- API 텍스트
  unique_key           STRING       -- 'fear_greed|<unix_ts>'
) USING DELTA
PARTITIONED BY (dt)
""")

src = spark.table(BRONZE)

silver_df = (
  src.select(
      F.col("event_time").cast("timestamp").alias("event_time"),
      F.col("dt").cast("date").alias("dt"),
      F.col("index_value").cast("int").alias("index_value"),
      F.col("value_classification").alias("value_classification"),
      F.col("time_until_update").alias("time_until_update"),
      F.col("unique_key").alias("unique_key")
  )
  .withColumn("ts_unix", F.unix_timestamp("event_time").cast("long"))
  .dropDuplicates(["unique_key"])
  .repartition("dt")
)

t = DeltaTable.forName(spark, SILVER)
# updateAll을 써도 s에 ts_unix가 있으므로 안전
(t.alias("t")
 .merge(silver_df.alias("s"), "t.unique_key = s.unique_key AND t.dt = s.dt")
 .whenMatchedUpdateAll()
 .whenNotMatchedInsertAll()
 .execute())

print(f"[FNG SILVER] upsert complete -> {SILVER}")
